In [1]:
import sqlite3, re

# change this to the relevant config file
from novartis_config import req_sub_dict

In [2]:
class DataBase():
    def __init__(self, database, req_sub_dict):
        # connect to database
        conn = sqlite3.connect(database+'.sqlite')
        globals()['cur'] = conn.cursor()
        self.req_sub_dict = req_sub_dict
        
    def sub_reqs(self, string, context_details):
        # searches through entity placeholders in text and substitutes relevant context to the conversation
        for requirement in self.req_sub_dict.keys():
            if requirement in string:
                exec(self.req_sub_dict[requirement][0], locals())
                exec(self.req_sub_dict[requirement][1], locals())
                string = re.sub(requirement, locals()['sub_data'], string)
        return string
    
    def query(self, query, answer, context_details: dict):
        query = self.sub_reqs(query, context_details)
        cur.execute(query)
        try:
            exec(answer, globals())
            answer_text = globals()['answer_text']
        except Exception as e:
            print(e)
            answer_text = 'sorry, there is no information for that.'
        return answer_text

In [3]:
database = 'NovartisQA'

In [4]:
db = DataBase(database, req_sub_dict)

In [5]:
def get_question_data(question_id):
    cur.execute('SELECT * FROM Questions WHERE id = ?', (question_id, ))
    question_data = cur.fetchall()[0]
    # sql query
    query = question_data[5]
    # answer
    answer = question_data[6]
    return query, answer

In [6]:
query, answer = get_question_data(3)

In [7]:
from synthetic_pharma_data import get_settings

In [8]:
context_details = get_settings()

In [9]:
context_details

{'<age_group>': 'child',
 '<condition>': 'pregnant',
 '<delivery>': 'oral',
 '<drug_name>': 'Paclitaxel',
 '<strength>': '10',
 '<symptom>': 'depression',
 '<units>': 'grams'}

In [10]:
db.query(query, answer, context_details)

Note: Contact of the undiluted concentrate with plasticized PVC equipment or devices used to prepare solutions for infusion is not recommended. In order to minimize patient exposure to the plasticizer DEHP [di-(2-ethylhexyl)phthalate], which may be leached from PVC infusion bags or sets, diluted paclitaxel solutions should be stored in bottles (glass, polypropylene) or plastic bags (polypropylene, polyolefin) and administered through polyethylene-lined administration sets. All patients should be premedicated prior to paclitaxel administration in order to prevent severe hypersensitivity reactions. Such premedication may consist of dexamethasone 20 mg PO administered approximately 12 and 6 hou


'Note: Contact of the undiluted concentrate with plasticized PVC equipment or devices used to prepare solutions for infusion is not recommended. In order to minimize patient exposure to the plasticizer DEHP [di-(2-ethylhexyl)phthalate], which may be leached from PVC infusion bags or sets, diluted paclitaxel solutions should be stored in bottles (glass, polypropylene) or plastic bags (polypropylene, polyolefin) and administered through polyethylene-lined administration sets. All patients should be premedicated prior to paclitaxel administration in order to prevent severe hypersensitivity reactions. Such premedication may consist of dexamethasone 20 mg PO administered approximately 12 and 6 hou'

In [11]:
ln = """bye	"[""data = 'no problem' "", 'answer_text = data', [], []]"
check_update	"[""data = 'ok, is there anything else to update?' "", 'answer_text = data', [], []]"
information_retrieval	"[""data = 'ok, let me look in to that' "", 'answer_text = data', [], []]"
information_retrieval+dose_question	"['cur.execute(""SELECT Drugs.name, dose, units, Periodicities.periodicity                         FROM Dosages JOIN Drugs JOIN Conditions JOIN Periodicities JOIN Delivery JOIN Ages                         ON Drugs.id = drug_id AND Periodicities.id = period_id AND age_id = Ages.id                         WHERE Drugs.name = \'<drug_name>\' AND Ages.age_group = \'<age_group>\' AND Delivery.method = \'<delivery>\'                          AND strength = \'<strength>\' "")', ""data = cur.fetchall(); answer_text = 'The dosage for %s is %s %s per %s' % data[0]"", [3, 0, 2, 4], []]"
information_retrieval+ingredient_question	"['cur.execute(""SELECT ingredients                         FROM Drug_versions JOIN Drugs JOIN Delivery                           ON Drugs.id = drug_id AND Delivery.id = delivery_id                          WHERE Drugs.name = \'<drug_name>\' AND Delivery.method = \'<delivery>\'"")', 'answer_text = cur.fetchall()[0][0]', [3, 2], []]"
information_retrieval+should_question	"['cur.execute(""SELECT advice                          FROM Contraindications JOIN Drugs JOIN Conditions                          ON drug_id = Drugs.id AND condition_id = Conditions.id                         WHERE Drugs.name = \'<drug_name>\' AND Conditions.condition = \'<condition>\'"")', 'answer_text = cur.fetchall()[0][0]', [3, 1], []]"
information_retrieval+side_effect_question	"['cur.execute(""SELECT Symptoms.symptom, Severities.severity, Drugs.name, advice                         FROM Complaint JOIN Drugs JOIN Symptoms JOIN Severities                          ON Drugs.id = drug_id AND Symptoms.id = symptom_id AND Severities.id = severity_id                         WHERE Drugs.name = \'<drug_name>\' AND Symptoms.symptom = \'<symptom>\' "")', ""data = cur.fetchall(); answer_text = '%s is a %s side effect of %s. %s' % data[0]"", [3, 5], []]"
information_retrieval+store_question	"['cur.execute(""SELECT storage                         FROM Drug_versions JOIN Drugs JOIN Delivery                           ON Drugs.id = drug_id AND Delivery.id = delivery_id                          WHERE Drugs.name = \'<drug_name>\' AND Delivery.method = \'<delivery>\'"")', 'answer_text = cur.fetchall()[0][0]', [3, 2], []]"
information_retrieval+usage_question	"['cur.execute(""SELECT instructions                          FROM Drug_versions JOIN Drugs JOIN Delivery                          ON drug_id = Drugs.id AND delivery_id = Delivery.id                         WHERE Drugs.name = \'<drug_name>\' AND Delivery.method = \'<delivery>\' "")', 'answer_text = cur.fetchall()[0][0]', [3, 2], []]"
information_retrieval+what_question	"['cur.execute(""SELECT indication FROM Drugs WHERE name = \'<drug_name>\'"")', 'answer_text = cur.fetchall()[0][0]', [3], []]"
no_information	"[""data = 'sorry, there is no information for that.' "", 'answer_text = data', [], []]"
request_information+<age_group>	"[""data = 'for which age group?'"", 'answer_text = data', [], [0]]"
request_information+<condition>	"[""data = '<drug_name> may not be recommended for people with <contraindications>, do you have any of these?'"", 'answer_text = data', [], [1]]"
request_information+<delivery>	"[""data = 'what version of <drug_name> is that, for example: oral, intravenous or subcutaneous?'"", 'answer_text = data', [], [2]]"
request_information+<strength>	"[""data = 'what is the strength of the drug?'"", 'answer_text = data', [], [4]]"
request_information+<symptom>	"[""data = 'these are some common side effects of <drug_name>: <side_effects>, are you seeing any other symptoms?'"", 'answer_text = data', [], [5]]"
request_information+<units>	"[""data = 'What are the units of the drug?'"", 'answer_text = data', [], [6]]"
welcomemsg	"[""data = 'hello, what can I help you with today?' "", 'answer_text = data', [], []]"
"""

In [12]:
act, template = ln.strip('\n').split('\t', 1)

In [13]:
act

'bye'

In [14]:
template

'"[""data = \'no problem\' "", \'answer_text = data\', [], []]"\ncheck_update\t"[""data = \'ok, is there anything else to update?\' "", \'answer_text = data\', [], []]"\ninformation_retrieval\t"[""data = \'ok, let me look in to that\' "", \'answer_text = data\', [], []]"\ninformation_retrieval+dose_question\t"[\'cur.execute(""SELECT Drugs.name, dose, units, Periodicities.periodicity                         FROM Dosages JOIN Drugs JOIN Conditions JOIN Periodicities JOIN Delivery JOIN Ages                         ON Drugs.id = drug_id AND Periodicities.id = period_id AND age_id = Ages.id                         WHERE Drugs.name = \'<drug_name>\' AND Ages.age_group = \'<age_group>\' AND Delivery.method = \'<delivery>\'                          AND strength = \'<strength>\' "")\', ""data = cur.fetchall(); answer_text = \'The dosage for %s is %s %s per %s\' % data[0]"", [3, 0, 2, 4], []]"\ninformation_retrieval+ingredient_question\t"[\'cur.execute(""SELECT ingredients                       

In [15]:
import copy, re

In [16]:
class DualTemplate():

    def __init__(self, default="", dontcare=""):
        self.default = default
        self.dontcare = dontcare

    @property
    def dontcare_slots(self):
        default_slots = self._slots(self.default)
        dontcare_slots = self._slots(self.dontcare)
        return default_slots - dontcare_slots

    @staticmethod
    def _slots(text):
        return set(re.findall('#(\w+)', text))

    @classmethod
    def from_str(cls, s):
        return cls(*s.split('\t', 1))

    def update(self, default="", dontcare=""):
        self.default = self.default or default
        self.dontcare = self.dontcare or dontcare

    def __contains__(self, t):
        return t.default and (t.default == self.default)\
                or t.dontcare and (t.dontcare == self.dontcare)

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.default == other.default)\
                    and (self.dontcare == other.dontcare)
        return False

    def __hash__(self):
        """Override the default hash behavior (that returns the id)"""
        return hash(self.default + '\t' + self.dontcare)

    def __str__(self):
        return self.default + '\t' + self.dontcare

    def generate_text(self, slots):
        t = copy.copy(self.default)
        if isinstance(slots, dict):
            slots = slots.items()
        dontcare_slots = (s[0] for s in slots if s[1] == 'dontcare')
        if self.dontcare and self.dontcare_slots.issubset(dontcare_slots):
            t = copy.copy(self.dontcare)
        for slot, value in slots:
            t = t.replace('#' + slot, value, 1)
        if t:
            t = t[0].upper() + t[1:]
        return t

In [17]:
s = "Sorry, I can't hear you."

In [18]:
DualTemplate.from_str(s).generate_text([])

"Sorry, I can't hear you."

In [19]:
DualTemplate.from_str(s).default

"Sorry, I can't hear you."

In [20]:
DualTemplate.from_str(s).dontcare

''

In [2]:
action = "data = 'no problem'"
answer = "answer_text = data"
exec(action, locals())

In [3]:
exec(answer, locals())

In [4]:
locals()['answer_text']

'no problem'

In [6]:
import os

In [1]:
s="a b c"

In [2]:
s.split(" ", 2)

['a', 'b', 'c']